In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import os
import json
import matplotlib.pyplot as plt
import PIL
import read_datasets as rd

from importlib import reload


import base64
import pickle
import collections
import itertools, collections
from sklearn.metrics import mean_squared_error
import cv2 
import math

In [2]:
reload(rd)

<module 'read_datasets' from '/home/xiaoyuz1/amazon_turk/read_datasets.py'>

In [3]:
import torch.nn.utils.rnn as rnn
import torch

In [4]:
np.set_printoptions(precision=4, suppress=True)

In [5]:
dfo = pd.read_csv('/raid/xiaoyuz1/amazon_turk/df_all.csv')
dfo['no_punc_1'] = dfo.no_punc_1.apply(lambda x: [str(y).strip()[1:-1] for y in x[1:-1].split(',')])

dfp = pd.read_csv('/raid/xiaoyuz1/amazon_turk/df_all_pair.csv')
dfp['no_punc_1'] = dfp.no_punc_1.apply(lambda x: [str(y).strip()[1:-1] for y in x[1:-1].split(',')])
dfp['no_punc_2'] = dfp.no_punc_2.apply(lambda x: [str(y).strip()[1:-1] for y in x[1:-1].split(',')])

## use pretrained word features

In [ ]:
import clip
device = "cuda"
model, preprocess = clip.load("ViT-B/32", device=device, jit=False) #Must set jit=False for training

In [ ]:
root_folder = '/raid/xiaoyuz1/clip_model_checkpoint'
torch_path_name = os.path.join(root_folder, "pleasant-tree-10.pt")
checkpoint = torch.load(torch_path_name)
print(checkpoint.keys())
args = checkpoint['args']

model.load_state_dict(checkpoint['model_state_dict'])

## test out primitive selector

In [15]:
test_file = "/raid/xiaoyuz1/primitive_selector_training_data/july_18_test.pkl"
data_raw = pickle.load(open(test_file, "rb"))

In [32]:
info = data_raw[72]
"{} {}".format(info['processed'], CONST.face_parts_idx_dict_doodler[info['part']])

'large eyes'

In [14]:
for split in ["train","test","dev"]:
    test_file = f"/raid/xiaoyuz1/primitive_selector_training_data/july_15_{split}.pkl"

    data_raw = pickle.load(open(test_file, "rb"))
    data_extended = []
    for info in data_raw:
        t1 = info["raw"]
        info["raw"] = " ".join(t1.split(" ")[:-1])
        t2 = info["processed"]
        info["processed"] = " ".join(t2.split(" ")[:-1])
        a,b,c,d,e,f,_,_,_ = info["M"]
        info.update({
            "p0" : a,
            "p1" : b,
            "p2" : c,
            "p3" : d,
            "p4" : e,
            "p5" : f,
        })
        data_extended.append(info)
    with open(f"/raid/xiaoyuz1/primitive_selector_training_data/july_18_{split}.pkl", 'wb+') as f:
        pickle.dump(data_extended, f)

In [6]:
import primitive_selector as ps
reload(ps)

<module 'primitive_selector' from '/home/xiaoyuz1/amazon_turk/primitive_selector.py'>

In [7]:
import sys
sys.argv = ['']
args = ps.get_args()

In [8]:
hp = ps.HParams()
args_dict = vars(args)
for k,v in args_dict.items():
    if k == "parameter_names":
        for j,name in enumerate(v):
            hp.parameter_names[j] = name
        continue
    if hasattr(hp, k):
        setattr(hp, k,v)
vocab = ps.preprocess_dataset_language(args.train_file)
hp.vocab_size = len(vocab)

In [9]:
TEMPLATE_DICT = {
    'arc' : lambda n : rd.generate_arc(n1=n, radius=1, x0=0, y0=0, template_size=1),
    'circle' : lambda n : rd.generate_circle(n1=n, radius=1, x0=0, y0=0, template_size=1),
    'square' : lambda n : rd.generate_square(n1=n, template_size=2),
    'semicircle' : lambda n : rd.generate_semicircle(n1=n, radius=1, x0=0, y0=0, template_size=1),
    'zigzag1' : lambda n : rd.generate_zigzag(n1=n, num_fold=1, side_length=1,template_size=1),
}
templates = {}
for i,(k,v) in enumerate(TEMPLATE_DICT.items()):
    arr = v(args.num_sampled_points)
    templates[i] = (k,arr)

In [10]:
vocab["eyes"]

1296

In [ ]:
train_sequences = pickle.load(open(args.train_seq_file, 'rb'))

In [ ]:
test_sequences = pickle.load(open(args.test_seq_file, 'rb'))

In [ ]:
sequences_extended = []
for data, template, result in test_sequences:
    gt_img = np.asarray(rd.render_img([result], line_diameter=3))
    sequences_extended.append((data, template, result, gt_img))
with open(args.test_seq_file, 'wb+') as f:
    pickle.dump(sequences_extended, f)

In [ ]:
print(len(train_sequences), len(test_sequences))

In [11]:
trainer = ps.Trainer(hp, args, vocab, templates)

In [30]:
trainer.test_dataset.data_raw[116], trainer.test_dataset[116]

({'category': 'face',
  'image_idx': 172,
  'part': 0,
  'raw': 'pupilless',
  'processed': 'pupilless',
  'primitive_type': 1,
  'M': array([-30.6854, -33.7167, 122.937 ,  17.6897, -12.1063,  91.9407,
           0.    ,   0.    ,   1.    ]),
  'error': 51.11977070676713,
  'split': 'test',
  'theta': 2.6186427544429574,
  'sx': 35.41919839344829,
  'sy': 27.327690743533438,
  'hx': 0.6539971055000693,
  'tx': 122.93695230331444,
  'ty': 91.9407071367543,
  'p0': -30.68541047399987,
  'p1': -33.71665550947047,
  'p2': 122.93695230331444,
  'p3': 17.68969188190086,
  'p4': -12.10632828731588,
  'p5': 91.9407071367543},
 (tensor([1296]),
  tensor(1),
  tensor([  2.6186,  35.4192,  27.3277,   0.6540, 122.9370,  91.9407]),
  tensor(116)))

In [34]:
"eyes" in vocab.keys()

True

In [35]:
sys.float_info.max

1.7976931348623157e+308

In [24]:
trainer.test_dataset.vocab_reverse[142]

'hair'

In [ ]:
checkpoint = torch.load('/raid/xiaoyuz1/doodler_model_checkpoint/dulcet-rain-8/55000.pt')

In [ ]:
trainer.model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
len(trainer.test_dataset_loader)

In [18]:
for batch_idx, (description_ts, primitive_types, affine_paramss, indices) in \
    enumerate(trainer.test_dataset_loader):
    if batch_idx == 1:
        break

In [20]:
indices

tensor([117, 122,  91,  94, 107, 118,  64,  65,  66,  67,  69,  71,  76,  77,
         82,  83,  84,  85,  88,  90,  92,  93,  95,  96,  97,  98,  99, 100,
        101, 103, 105, 108, 110, 111, 112, 113, 114, 119, 120, 121, 124, 125,
        126,  68,  70,  72,  73,  74,  75,  78,  79,  80,  81,  86,  87,  89,
        104, 106, 109, 115, 123, 127, 102, 116])

In [29]:
description_ts

[tensor([187, 196, 255,   5, 410]),
 tensor([ 36, 403, 360, 361, 410]),
 tensor([  5, 175,  88, 410]),
 tensor([   5,   45,   46, 1296]),
 tensor([  19,  595,  169, 1296]),
 tensor([280, 229,   4, 196]),
 tensor([563,  13, 142]),
 tensor([  8,  33, 196]),
 tensor([ 89,  88, 410]),
 tensor([  3,  81, 196]),
 tensor([ 45,  10, 139]),
 tensor([  7, 290, 196]),
 tensor([150, 158, 142]),
 tensor([243,  10, 410]),
 tensor([   7,  242, 1296]),
 tensor([268,  74, 142]),
 tensor([ 30,  85, 142]),
 tensor([  41,   20, 1296]),
 tensor([252, 229, 410]),
 tensor([297,   9, 410]),
 tensor([ 40,  54, 142]),
 tensor([256,  79, 196]),
 tensor([  37,  427, 1296]),
 tensor([  5,  32, 139]),
 tensor([ 55, 329, 139]),
 tensor([202,  73, 142]),
 tensor([191,  84, 139]),
 tensor([912,  46, 139]),
 tensor([  6,  46, 142]),
 tensor([ 39,   4, 196]),
 tensor([ 10,  34, 410]),
 tensor([ 10,  36, 196]),
 tensor([ 19,   4, 196]),
 tensor([297,  34, 139]),
 tensor([ 167,  150, 1296]),
 tensor([ 10,  22, 410]),
 ten

In [25]:
rnn.pack_sequence(description_ts)

PackedSequence(data=tensor([ 187,   36,    5,    5,   19,  280,  563,    8,   89,    3,   45,    7,
         150,  243,    7,  268,   30,   41,  252,  297,   40,  256,   37,    5,
          55,  202,  191,  912,    6,   39,   10,   10,   19,  297,  167,   10,
         234,    7,  551,   31,    8, 1128,   81,   81,  268,    7,   10,    5,
          27,   19,    3,   25,   17,   71,   27,   38,  147,   17,   41,  206,
          55,   19,  142, 1296,  196,  403,  175,   45,  595,  229,   13,   33,
          88,   81,   10,  290,  158,   10,  242,   74,   85,   20,  229,    9,
          54,   79,  427,   32,  329,   73,   84,   46,   46,    4,   34,   36,
           4,   34,  150,   22,  401,   19,   42,   79,  119,  157,    6,  139,
         142, 1296, 1296,  139,  142, 1296,  139,  139, 1296, 1296,  142, 1296,
        1296,  196, 1296,  139,  139,  196,  255,  360,   88,   46,  169,    4,
         142,  196,  410,  196,  139,  196,  142,  410, 1296,  142,  142, 1296,
         410,  410, 

In [ ]:
description_ts_packed = rnn.pack_sequence(description_ts)
description_ts_packed, primitive_types, affine_paramss = description_ts_packed.to(trainer.device), primitive_types.to(trainer.device), affine_paramss.to(trainer.device)

prim_pred, normal_dists, pi_dists = trainer.model(description_ts_packed)  
prim_types = torch.argmax(prim_pred, dim=1)
param_samples = trainer.sample_parameters(normal_dists, pi_dists) # each: N x 1
np.random.seed(123)
plot_indices = np.random.choice(len(indices), trainer.args.num_visualize, replace=False)
print(plot_indices)

In [ ]:
trainer.templates.keys()

In [ ]:
trainer.model.primitive_fc

In [ ]:
trainer.calculate_metric(description_ts, indices, prim_types, param_samples, plot_indices)

In [ ]:
trainer.test_meter.count

In [ ]:
trainer.test_meter.metric_dict

In [ ]:
samples = trainer.sample_parameters(normal_dists, pi_dists) # each: N x 1
np.random.seed(123)
plot_indices = np.random.choice(len(indices), trainer.args.num_visualize, replace=False)
dataset_indices = [indices[j].item() for j in plot_indices]

In [ ]:
idx = plot_indices[0]
data_idx = dataset_indices[0]
prim_type = prim_types[idx].item()
info = trainer.test_dataset.data_raw[data_idx]
data, template, result = trainer.test_sequences[data_idx]

In [ ]:
img1 = rd.render_img(
    [result], 
    img_path=None, 
    show=False,
    side=256,
    line_diameter=3,
    padding=0,
    bg_color=(0,0,0),
    fg_color=(1,1,1),
    original_side = 256.,
    convert = True,
)
img1

In [ ]:
pred_info = {}
pred_params = [sample[idx].item() for sample in samples]
pred_info["theta"],pred_info["sx"],pred_info["sy"],pred_info["hx"],pred_info["tx"],pred_info["ty"] = \
    pred_params
pred_M = ps.get_affine_transformation(pred_info)

pred_template_name, pred_template = trainer.templates[int(prim_type)]
gt_template_name,_ = trainer.templates[int(info["primitive_type"])]
if trainer.args.use_projective:
    pred_template_pred_param = cv2.perspectiveTransform(pred_template, pred_M).reshape(-1,2)
else:
    pred_template_pred_param = cv2.transform(np.array([pred_template]).astype(np.float32), pred_M)[0][:,:-1]


In [ ]:
info, pred_info

In [ ]:
np.degrees(info["theta"]),np.degrees(pred_info["theta"])

In [ ]:
def mod(a, n):
    return (a % n + n) % n

def diff_angle(targetA, sourceA):
    a = targetA - sourceA
    a = mod(a+np.pi, 2 * np.pi) - np.pi
    return a

np.degrees(diff_angle(pred_info["theta"], info["theta"]))

In [ ]:
img2 = rd.render_img(
    [pred_template_pred_param], 
    img_path=None, 
    show=False,
    side=256,
    line_diameter=3,
    padding=0,
    bg_color=(0,0,0),
    fg_color=(1,1,1),
    original_side = 256.,
    convert = True,
)
img2

In [ ]:
img1 = np.asarray(img1)
img2 = np.asarray(img2)



In [ ]:
# 

In [ ]:
def calculate_psnr(img1, img2, max_value=255):
    """"Calculating peak signal-to-noise ratio (PSNR) between two images."""
    mse = np.mean((np.array(img1, dtype=np.float32) - np.array(img2, dtype=np.float32)) ** 2)
    if mse == 0:
        return 100
    return 20 * np.log10(max_value / (np.sqrt(mse)))

calculate_psnr(img1,img2)

In [ ]:
type(img1.astype(np.float32)[0][0])

In [ ]:
np.mean((np.array(img1, dtype=np.float32) - np.array(img2, dtype=np.float32)) ** 2)

In [ ]:
from skimage.metrics import structural_similarity as ssim
img1[img1 < 1] = 1
img1[img1 == 255] = 0

img2[img2 < 1] = 1
img2[img2 == 255] = 0

ssim(img1, img2, multichannel=False, data_range=255)

In [ ]:
prim_types.shape

## fitting primitives

In [ ]:
reload(rd)

In [ ]:
w = h = 256
TEMPLATE_DICT = {
    'arc' : lambda n : rd.generate_arc(n1=n, radius=1, x0=0, y0=0, template_size=1),
    'circle' : lambda n : rd.generate_circle(n1=n, radius=1, x0=0, y0=0, template_size=1),
    'square' : lambda n : rd.generate_square(n1=n, template_size=2),
    'semicircle' : lambda n : rd.generate_semicircle(n1=n, radius=1, x0=0, y0=0, template_size=w),
    'zigzag1' : lambda n : rd.generate_zigzag(n1=n, num_fold=1, side_length=1,template_size=1),
}

In [ ]:
b_spline_num_sampled_points = num_sampled_points = n = 200
use_projective = False

In [ ]:
templates = {}
template_dict_processed = {}
for i,(k,v) in enumerate(TEMPLATE_DICT.items()):
    arr = v(b_spline_num_sampled_points)
    template_dict_processed[k] = arr
    templates[i] = (k,arr)

In [ ]:
rd.plot_primitives(template_dict_processed,w=2, h=2,num_pngs_per_row = 1, row_figsize = 10, column_figsize = 10)

In [ ]:
import primitive_selector as ps
reload(ps)
all_data = ps.prepare_data(dfo, templates, num_sampled_points = 200, use_projective = False)

In [ ]:
with open("/raid/xiaoyuz1/primitive_selector_training_data/july_15_all.pkl", "wb+") as f:
    pickle.dump(all_data, f)

In [ ]:
for t in ["train", "dev", "test"]:
    data_split = [x for x in all_data if x["split"] == t]
    data_split_dict = dict(zip(range(len(data_split)), data_split))
    with open(f"/raid/xiaoyuz1/primitive_selector_training_data/july_15_{t}.pkl", "wb+") as f:
        pickle.dump(data_split, f)

In [ ]:
all_data = pickle.load(open("/raid/xiaoyuz1/primitive_selector_training_data/july_15_all.pkl", "rb"))
df = pd.DataFrame(all_data)

In [ ]:
df

In [ ]:
data,template,result = all_sequences[100]

fig, axs = plt.subplots(2, 1, figsize=(12,24)) 
axs[0].scatter(data[:,0], data[:,1], s=1, c='g')
axs[0].scatter(result[:,0], result[:,1], s=1, c='r')
axs[0].scatter(template[:,0], template[:,1], s=1, c='b')

axs[0].axis(xmin=-w,xmax=w)
axs[0].axis(ymin=h,ymax=-h)

plt.show()

In [ ]:
def plot_prepared_data(info, num_sampled_points = 200, use_projective=False):
    fig, axs = plt.subplots(2, 1, figsize=(12,24)) 
    if info['category'] == 'face':
        drawing_raw = CONST.face_json['train_data'][info['image_idx']]
        part_indices = list(CONST.face_parts_idx_dict_doodler.keys())
    else:
        drawing_raw = CONST.angel_json['train_data'][info['image_idx']]
        part_indices = list(CONST.angel_parts_idx_dict_doodler.keys())

    parts = rd.process_quickdraw_to_part_convex_hull(
        drawing_raw,
        part_indices,
        b_spline_num_sampled_points=num_sampled_points,
    )
    data = parts[info['part']]
    template = templates[info['primitive_type']][1]
    transform_mat = np.asarray(info['M']).astype(float).reshape(3,3)
#     axs[0].scatter(template[:,0], template[:,1], s=1, c='b')
    
    if use_projective:
        result = cv2.perspectiveTransform(template, transform_mat).reshape(-1,2)
    else:
        result = cv2.transform(np.array([template]).astype(
            np.float32), transform_mat)[0][:,:-1]
    
    
    axs[0].scatter(result[:,0], result[:,1], s=1, c='r')
    
#     theta, scale_mat, shear_mat = rd.decompose_affine(transform_mat)
#     T_rotate = rd.get_rotation_matrix(theta)
#     T_scale = rd.get_scale_matrix(scale_mat[0][0], scale_mat[1][1])
#     T_shear = rd.get_shear_matrix(shear_mat[0][1], shear_mat[1][0])
#     tx,ty = transform_mat[0][2],transform_mat[1][2]
#     T_translate = rd.get_translation_matrix(tx,ty)
    
#     result1 = cv2.transform(np.array([template]).astype(
#             np.float32), T_rotate)[0][:,:-1]
#     axs[0].scatter(result1[:,0], result1[:,1], s=1, c='m')
    
#     result2 = cv2.transform(np.array([template]).astype(
#             np.float32), T_rotate @ T_scale @ T_shear)[0][:,:-1]
#     axs[0].scatter(result2[:,0], result2[:,1], s=1, alpha=0.5, c='pink')

#     result3 = cv2.transform(np.array([template]).astype(
#             np.float32), T_translate @ T_rotate @ T_scale @ T_shear)[0][:,:-1]
#     axs[0].scatter(result3[:,0], result3[:,1], s=1, c='lime', alpha=0.5)
    
    axs[0].scatter(data[:,0], data[:,1], alpha=0.5, s=1, c='lime')

    axs[0].axis(xmin=-w,xmax=w)
    axs[0].axis(ymin=h,ymax=-h)
    axs[1].axis(xmin=-w,xmax=w)
    axs[1].axis(ymin=h,ymax=-h)

    
    
    plt.show()

In [ ]:
plot_prepared_data(df.iloc[123])

In [ ]:
df.iloc[123]

### plot GMM

In [ ]:
M_all = np.asarray(df.M.to_list())

In [ ]:
from sklearn.mixture import GaussianMixture

X = np.asarray(df.sx.to_list()).reshape(-1, 1)
# M_all[:,0].reshape(-1, 1)
print(X.min(), X.max())

N = np.arange(1, 11)
models = [None for i in range(len(N))]

for i in range(len(N)):
    models[i] = GaussianMixture(N[i]).fit(X)

# compute the AIC and the BIC
AIC = [m.aic(X) for m in models]
BIC = [m.bic(X) for m in models]


#------------------------------------------------------------
# Plot the results
#  We'll use three panels:
#   1) data + best-fit mixture
#   2) AIC and BIC vs number of components
#   3) probability that a point came from each component

fig = plt.figure(figsize=(15, 20))
fig.subplots_adjust(left=0.12, right=0.97,
                    bottom=0.21, top=0.9, wspace=0.5)


# plot 1: data + best-fit mixture
ax = fig.add_subplot(311)
M_best = models[np.argmin(AIC)]

x = np.linspace(X.min(), X.max(), 1000)
logprob = M_best.score_samples(x.reshape(-1, 1))
responsibilities = M_best.predict_proba(x.reshape(-1, 1))
pdf = np.exp(logprob)
pdf_individual = responsibilities * pdf[:, np.newaxis]

ax.hist(X, 30, density=True, histtype='stepfilled', alpha=0.4)
ax.plot(x, pdf, '-k')
ax.plot(x, pdf_individual, '--k')
ax.text(0.04, 0.96, "Best-fit Mixture",
        ha='left', va='top', transform=ax.transAxes)
ax.set_xlabel('$x$')
ax.set_ylabel('$p(x)$')


# plot 2: AIC and BIC
ax = fig.add_subplot(312)
ax.plot(N, AIC, '-k', label='AIC')
ax.plot(N, BIC, '--k', label='BIC')
ax.set_xlabel('n. components')
ax.set_ylabel('information criterion')
ax.legend(loc=2)


# plot 3: posterior probabilities for each component
ax = fig.add_subplot(313)

p = responsibilities
p = p[:, (1, 0, 2)]  # rearrange order so the plot looks better
p = p.cumsum(1).T

ax.fill_between(x, 0, p[0], color='gray', alpha=0.3)
ax.fill_between(x, p[0], p[1], color='gray', alpha=0.5)
ax.fill_between(x, p[1], 1, color='gray', alpha=0.7)
ax.set_xlim(-6, 6)
ax.set_ylim(0, 1)
ax.set_xlabel('$x$')
ax.set_ylabel(r'$p({\rm class}|x)$')

ax.text(-5, 0.3, 'class 1', rotation='vertical')
ax.text(0, 0.5, 'class 2', rotation='vertical')
ax.text(3, 0.3, 'class 3', rotation='vertical')

plt.show()


In [ ]:
print(M_best.weights_)
print(M_best.means_)
print(M_best.covariances_)

### code

In [ ]:
df.iloc[110]

In [ ]:
plot_prepared_data(df.iloc[110])

In [ ]:
drawing_arr = CONST.face_json['train_data']

In [ ]:
reload(rd)
from tqdm import tqdm

M_arr = collections.defaultdict(lambda : np.zeros(9))
M_name = collections.defaultdict(lambda : " ")
M_mse = collections.defaultdict(float)


for idx in tqdm(range(len(drawing_arr))):
# for idx in range(len(drawing_arr)):
#     print(idx)
    
    strokes_spline_fitted = rd.process_quickdraw_to_part_convex_hull(
        drawing_arr[idx],
        list(CONST.face_parts_idx_dict.keys()),
        b_spline_num_sampled_points=n,
    )

    for _, (part_type, sketch_data) in enumerate(strokes_spline_fitted.items()):
        
        mat, prim_name, prim_mse = rd.get_transform_smallest_mse(
            sketch_data, template_dict_processed, use_projective=use_projective,
        )
        M_arr[(idx, part_type)] = mat
        M_name[(idx, part_type)] = prim_name
        M_mse[(idx, part_type)] = prim_mse
        


In [ ]:
with open('/raid/xiaoyuz1/SPG_Face_Part_256_projective_{}_mse.pkl'.format(use_projective), "wb+") as f:
    pickle.dump((dict(M_arr), dict(M_name), dict(M_mse)), f)

In [ ]:
'/raid/xiaoyuz1/SPG_Face_Part_256_projective_{}_mse.pkl'.format(use_projective)

In [ ]:
line_dict = dict(filter(lambda x: x[1] == 'line', M_name.items()))
print(len(line_dict), line_dict)

In [ ]:
M_arr, M_name = None,None
M_mse = None
fname = '/raid/xiaoyuz1/SPG_Face_Part_256_projective_False_mse.pkl'
with open(fname, "rb") as f:
    M_arr, M_name, M_mse = pickle.load(f)

In [ ]:
mse_l = list(filter(lambda x: not math.isinf(x), list(M_mse.values())))
np.mean(mse_l)

In [ ]:
image_list = list(map(lambda x: int(x[0]), M_name.keys()))
stroke_list = list(map(lambda x: int(x[1]), M_name.keys()))
arr_list = list(M_arr.values())
name_list = list(M_name.values())
mse_list = list(M_mse.values())

data = {'image': image_list, 'stroke' : stroke_list, 'M' : arr_list, 'primitive' : name_list, 'mse' : mse_list}

for c_idx,c in enumerate(np.asarray(arr_list).T):
    if c_idx > 5:
        continue
    data['M{}'.format(str(c_idx))] = c

df = pd.DataFrame.from_dict(data)

In [ ]:
dfa = df[df.mse.apply(lambda x: not math.isinf(x))]

In [ ]:
dfa

In [ ]:
M_all = np.asarray(dfa.M.to_list())

---------------------------------------

In [ ]:
idx = 0
stroke_idx = 1

In [ ]:

rd.show_these_sketches(
    CONST.face_json,
    [idx], 
    [str(idx)], 
    [[]], 
    num_pngs_per_row = 6,
    row_figsize = 4,
    column_figsize = 4,
)



In [ ]:
strokes_spline_fitted = rd.process_quickdraw_to_part_convex_hull(
    drawing_arr[idx],
    list(CONST.face_parts_idx_dict.keys()),
    b_spline_num_sampled_points=n,
)

# strokes_spline_fitted = rd.process_quickdraw_to_stroke_no_normalize(
#     drawing_arr[idx], 
#     b_spline_num_sampled_points=200,
# )
# print(len(strokes_spline_fitted))

In [ ]:
from scipy.spatial import ConvexHull, convex_hull_plot_2d

parts_indices = list(CONST.face_parts_idx_dict.keys())

drawing_raw = np.asarray(drawing_arr[idx])
drawing_raw[:,0] = np.cumsum(drawing_raw[:,0], 0) + 25
drawing_raw[:,1] = np.cumsum(drawing_raw[:,1], 0) + 25

parts = []
part_idx_part = {}

for k in parts_indices:
    strokes = drawing_raw[drawing_raw[:,-1] == k]
    if len(strokes) < 1:
        continue
    parts.append(strokes)
    part_idx_part[k] = strokes[:,:2]

In [ ]:
M_arr_idx, M_name_idx, M_mse_idx = {},{},{}
for stroke_index, (part_type, data) in enumerate(strokes_spline_fitted.items()):
        
    mat, prim_name, prim_mse = rd.get_transform_smallest_mse(
        data, template_dict_processed, use_projective=use_projective,
    )
    M_arr_idx[(idx, part_type)] = mat
    M_name_idx[(idx, part_type)] = prim_name
    M_mse_idx[(idx, part_type)] = prim_mse

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

for k, M in M_arr_idx.items():
#     if k[1] != 3:
#         continue
    data = strokes_spline_fitted[k[1]]
    
    template = template_dict_processed[M_name_idx[(idx, k[1])]]
    Mm = M.reshape(3,3)
    
    result, mse = rd.get_transformed_template(template, data, Mm,projective=use_projective)
    ax1.scatter(result[:,0], result[:,1], label="transformed template", s=1, c='r')
    ax2.scatter(data[:,0], data[:,1], label="data", alpha=0.5, s=1, c='g')

# plt.legend()
# plt.xlim(0,w)
# plt.ylim(h,0)


ax1.axis(xmin=0,xmax=w)
ax1.axis(ymin=h,ymax=0)

ax2.axis(xmin=0,xmax=w)
ax2.axis(ymin=h,ymax=0)

plt.show()

In [ ]:
M_arr_idx, M_name_idx, M_mse_idx

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

for index, entry in dfa[dfa.image == idx].iterrows():
    data = strokes_spline_fitted[entry.stroke]
    
    template_func = TEMPLATE_DICT[entry.primitive]
    template = template_func(n)
    
    print(entry.M.reshape(3,3))
    
    result, mse = rd.get_transformed_template(template, data, entry.M.reshape(3,3),projective=use_projective)
    ax1.scatter(result[:,0], result[:,1], label="transformed template", s=1, c='r')
    ax2.scatter(data[:,0], data[:,1], label="data", alpha=0.5, s=1, c='g')
    orig_data = part_idx_part[int(entry.stroke)]
    ax2.plot(orig_data[:,0], orig_data[:,1], alpha=0.5, c='b')
    
#     ax1.plot(result[:,0], result[:,1], label="transformed template", c='r')
#     ax2.plot(data[:,0], data[:,1], label="data", alpha=0.5,c='g')

# plt.legend()
# plt.xlim(0,w)
# plt.ylim(h,0)

ax1.axis(xmin=0,xmax=w)
ax1.axis(ymin=h,ymax=0)

ax2.axis(xmin=0,xmax=w)
ax2.axis(ymin=h,ymax=0)

plt.show()

## old icp code

In [ ]:
import cv2
import numpy as np
import sys
from numpy.random import *

In [ ]:
def del_miss(indeces, dist, max_dist, th_rate = 0.8):
    th_dist = max_dist * th_rate
    return np.array([indeces[0][np.where(dist.T[0] < th_dist)]])


In [ ]:
def is_converge(Tr, scale):
    delta_angle = 0.0001
    delta_scale = scale * 0.0001
    
    min_cos = 1 - delta_angle
    max_cos = 1 + delta_angle
    min_sin = -delta_angle
    max_sin = delta_angle
    min_move = -delta_scale
    max_move = delta_scale
    
    return min_cos < Tr[0, 0] and Tr[0, 0] < max_cos and \
           min_cos < Tr[1, 1] and Tr[1, 1] < max_cos and \
           min_sin < -Tr[1, 0] and -Tr[1, 0] < max_sin and \
           min_sin < Tr[0, 1] and Tr[0, 1] < max_sin and \
           min_move < Tr[0, 2] and Tr[0, 2] < max_move and \
           min_move < Tr[1, 2] and Tr[1, 2] < max_move

In [ ]:
def icp(d1, d2, max_iterate = 100):
    src = np.array([d1.T], copy=True).astype(np.float32)
    dst = np.array([d2.T], copy=True).astype(np.float32)
    
    knn = cv2.KNearest()
    responses = np.array(range(len(d2[0]))).astype(np.float32)
    knn.train(src[0], responses)
        
    Tr = np.array([[np.cos(0), -np.sin(0), 0],
                   [np.sin(0), np.cos(0),  0],
                   [0,         0,          1]])

    dst = cv2.transform(dst, Tr[0:2])
    max_dist = sys.maxint
    
    scale_x = np.max(d1[0]) - np.min(d1[0])
    scale_y = np.max(d1[1]) - np.min(d1[1])
    scale = max(scale_x, scale_y)
       
    for i in range(max_iterate):
        ret, results, neighbours, dist = knn.find_nearest(dst[0], 1)
        
        indeces = results.astype(np.int32).T     
        indeces = del_miss(indeces, dist, max_dist)  
        
        T = cv2.estimateRigidTransform(dst[0, indeces], src[0, indeces], True)

        max_dist = np.max(dist)
        dst = cv2.transform(dst, T)
        Tr = np.dot(np.vstack((T,[0,0,1])), Tr)
        
        if (is_converge(T, scale)):
            break
        
    return Tr[0:2]

In [ ]:
if __name__ == "__main__":
    import matplotlib.pyplot as plt
    
    point_count = 100
    th = np.pi / 8
    move = np.array([[0.30], [0.5]])
    rnd_scale = 0.03
    x1 = np.linspace(0, 1.1, point_count)
    y1 = np.sin(x1 * np.pi)
    d1 = np.array([x1, y1])

    rot = np.array([[np.cos(th), -np.sin(th)], [np.sin(th), np.cos(th)]])
    rand = np.random.rand(2, point_count)*rnd_scale
    d2 = np.dot(rot, d1) + move
    d2 = np.add(d2, rand)

    plt.plot(d1[0], d1[1])
    plt.plot(d2[0], d2[1])
    plt.show()

In [ ]:
    ret = icp(d1, d2)
    
    plt.plot(d1[0], d1[1])
    dst = np.array([d2.T], copy=True).astype(np.float32)
    dst = cv2.transform(dst, ret)
    plt.plot(dst[0].T[0], dst[0].T[1])
    plt.show()
    
    print ret[0][0] * ret[0][0] + ret[0][1] * ret[0][1]
    print np.arccos(ret[0][0]) / 2 / np.pi * 360
    print np.arcsin(ret[0][1]) / 2 / np.pi * 360

    print ret

In [ ]:
def icp(a, b,
        max_time = 1
    ):
    import cv2
    import numpy
    import copy
    import pylab
    import time
    import sys
    import sklearn.neighbors
    import scipy.optimize



    def res(p,src,dst):
        T = numpy.matrix([[numpy.cos(p[2]),-numpy.sin(p[2]),p[0]],
        [numpy.sin(p[2]), numpy.cos(p[2]),p[1]],
        [0 ,0 ,1 ]])
        n = numpy.size(src,0)
        xt = numpy.ones([n,3])
        xt[:,:-1] = src
        xt = (xt*T.T).A
        d = numpy.zeros(numpy.shape(src))
        d[:,0] = xt[:,0]-dst[:,0]
        d[:,1] = xt[:,1]-dst[:,1]
        r = numpy.sum(numpy.square(d[:,0])+numpy.square(d[:,1]))
        return r

    def jac(p,src,dst):
        T = numpy.matrix([[numpy.cos(p[2]),-numpy.sin(p[2]),p[0]],
        [numpy.sin(p[2]), numpy.cos(p[2]),p[1]],
        [0 ,0 ,1 ]])
        n = numpy.size(src,0)
        xt = numpy.ones([n,3])
        xt[:,:-1] = src
        xt = (xt*T.T).A
        d = numpy.zeros(numpy.shape(src))
        d[:,0] = xt[:,0]-dst[:,0]
        d[:,1] = xt[:,1]-dst[:,1]
        dUdth_R = numpy.matrix([[-numpy.sin(p[2]),-numpy.cos(p[2])],
                            [ numpy.cos(p[2]),-numpy.sin(p[2])]])
        dUdth = (src*dUdth_R.T).A
        g = numpy.array([  numpy.sum(2*d[:,0]),
                        numpy.sum(2*d[:,1]),
                        numpy.sum(2*(d[:,0]*dUdth[:,0]+d[:,1]*dUdth[:,1])) ])
        return g
    
    def hess(p,src,dst):
        n = numpy.size(src,0)
        T = numpy.matrix([[numpy.cos(p[2]),-numpy.sin(p[2]),p[0]],
        [numpy.sin(p[2]), numpy.cos(p[2]),p[1]],
        [0 ,0 ,1 ]])
        n = numpy.size(src,0)
        xt = numpy.ones([n,3])
        xt[:,:-1] = src
        xt = (xt*T.T).A
        d = numpy.zeros(numpy.shape(src))
        d[:,0] = xt[:,0]-dst[:,0]
        d[:,1] = xt[:,1]-dst[:,1]
        dUdth_R = numpy.matrix([[-numpy.sin(p[2]),-numpy.cos(p[2])],[numpy.cos(p[2]),-numpy.sin(p[2])]])
        dUdth = (src*dUdth_R.T).A
        H = numpy.zeros([3,3])
        H[0,0] = n*2
        H[0,2] = numpy.sum(2*dUdth[:,0])
        H[1,1] = n*2
        H[1,2] = numpy.sum(2*dUdth[:,1])
        H[2,0] = H[0,2]
        H[2,1] = H[1,2]
        d2Ud2th_R = numpy.matrix([[-numpy.cos(p[2]), numpy.sin(p[2])],[-numpy.sin(p[2]),-numpy.cos(p[2])]])
        d2Ud2th = (src*d2Ud2th_R.T).A
        H[2,2] = numpy.sum(2*(numpy.square(dUdth[:,0])+numpy.square(dUdth[:,1]) + d[:,0]*d2Ud2th[:,0]+d[:,0]*d2Ud2th[:,0]))
        return H
    
    
    t0 = time.time()
    init_pose = (0,0,0)
    src = numpy.array([a.T], copy=True).astype(numpy.float32)
    dst = numpy.array([b.T], copy=True).astype(numpy.float32)
    Tr = numpy.array([[numpy.cos(init_pose[2]),-numpy.sin(init_pose[2]),init_pose[0]],
                   [numpy.sin(init_pose[2]), numpy.cos(init_pose[2]),init_pose[1]],
                   [0,                    0,                   1          ]])
    print("src",numpy.shape(src))
    print("Tr[0:2]",numpy.shape(Tr[0:2]))
    src = cv2.transform(src, Tr[0:2])
    p_opt = numpy.array(init_pose)
    T_opt = numpy.array([])
    error_max = sys.maxsize
    first = False
    while not(first and time.time() - t0 > max_time):
        distances, indices = sklearn.neighbors.NearestNeighbors(n_neighbors=1, algorithm='auto',p = 3).fit(dst[0]).kneighbors(src[0])
        p = scipy.optimize.minimize(res,[0,0,0],args=(src[0],dst[0, indices.T][0]),method='Newton-CG',jac=jac,hess=hess).x
        T  = numpy.array([[numpy.cos(p[2]),-numpy.sin(p[2]),p[0]],[numpy.sin(p[2]), numpy.cos(p[2]),p[1]]])
        p_opt[:2]  = (p_opt[:2]*numpy.matrix(T[:2,:2]).T).A       
        p_opt[0] += p[0]
        p_opt[1] += p[1]
        p_opt[2] += p[2]
        src = cv2.transform(src, T)
        Tr = (numpy.matrix(numpy.vstack((T,[0,0,1])))*numpy.matrix(Tr)).A
        error = res([0,0,0],src[0],dst[0, indices.T][0])

        if error < error_max:
            error_max = error
            first = True
            T_opt = Tr

    p_opt[2] = p_opt[2] % (2*numpy.pi)
    return T_opt, error_max


def main():
    import cv2
    import numpy
    import random
    import matplotlib.pyplot
    n1 = 100
    n2 = 75
    bruit = 1/10
    center = [random.random()*(2-1)*3,random.random()*(2-1)*3]
    radius = random.random()
    deformation = 2

    template = numpy.array([
        [numpy.cos(i*2*numpy.pi/n1)*radius*deformation for i in range(n1)], 
        [numpy.sin(i*2*numpy.pi/n1)*radius for i in range(n1)]
    ])

    data = numpy.array([
        [numpy.cos(i*2*numpy.pi/n2)*radius*(1+random.random()*bruit)+center[0] for i in range(n2)], 
        [numpy.sin(i*2*numpy.pi/n2)*radius*deformation*(1+random.random()*bruit)+center[1] for i in range(n2)]
    ])

    T,error = icp(data,template)
    dx = T[0,2]
    dy = T[1,2]
    rotation = numpy.arcsin(T[0,1]) * 360 / 2 / numpy.pi

    print("T",T)
    print("error",error)
    print("rotation°",rotation)
    print("dx",dx)
    print("dy",dy)

    result = cv2.transform(numpy.array([data.T], copy=True).astype(numpy.float32), T).T
    matplotlib.pyplot.plot(template[0], template[1], label="template")
    matplotlib.pyplot.plot(data[0], data[1], label="data")
    matplotlib.pyplot.plot(result[0], result[1], label="result: "+str(rotation)+"° - "+str([dx,dy]))
    matplotlib.pyplot.legend(loc="upper left")
    matplotlib.pyplot.axis('square')
    matplotlib.pyplot.show()

if __name__ == "__main__":
    main()